In this unit, we describe the procedure for training the transport mode detector. Further, we present the classification result for transport mode detector.

# Transport mode classifier

In [1]:
'''Import and initialize MongoClient'''
from scipy import integrate
import numpy as np
import pprint
import math


import subprocess
import os
import sys
import json
from pymongo import     MongoClient
from pathlib import Path

con = MongoClient()

import pprint

In [2]:
'''Import Libs'''
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score, recall_score, f1_score
from sklearn import tree
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.impute import SimpleImputer

In [3]:
'''Import project specific library'''
sys.path.append(os.path.join(os.getcwd(), 'LibCode'))
import ReadAcclGPSRecord
import SegmentsOtherThanStoppageSegments
import EarthaxisAcceleration
import EarthaxisAccelerationOnRawRecords
import ComputeFeaturesTransportMode
import FeaturesExtraction
import TransportModeFeatureHelper
import TMC_Helper

In [4]:
'''For updating the lib changes effects'''
import importlib
importlib.reload(ReadAcclGPSRecord)
importlib.reload(SegmentsOtherThanStoppageSegments)
importlib.reload(EarthaxisAcceleration)
importlib.reload(EarthaxisAccelerationOnRawRecords)
importlib.reload(ComputeFeaturesTransportMode)
importlib.reload(FeaturesExtraction)
importlib.reload(TransportModeFeatureHelper)
importlib.reload(TMC_Helper)

<module 'TMC_Helper' from '/home/pruthvish/JRF/GitVersion_APTS_Software_Np/code/LibCode/TMC_Helper.py'>

## Save in MongoDB
The transport mode detector is trained and validated on the trip records collected by data collection volunteers on their journey for various modes of transport. The accelerometer record is stored in the `RouteName = ISCON_PDPU_For_Transport_Mode` database of MongoDB. 

`ID_State_Position_Direction_RouteStart_RouteEnd_Date__Time`

`ID_Mode_Direction_RouteStart_RouteEnd_Date__Time`

In [5]:
def ReadAndSaveData(TransportModeDir, RouteName):
    '''
    input: The route name and dataset directory name
    
    output: None
    
    function: It fetches the data records placed in the specified data set directory 
    and save them in the MongoDB database
    '''
    
    '''E.g., BM, CM, Bike, Car'''
    for fileName in [f for f in os.listdir(os.path.join(TransportModeDir,RouteName))]:
        FileNameSplitted = fileName.split('_')
        DeviceID = FileNameSplitted[0]
        Mode = FileNameSplitted[1]
        ReadAcclGPSRecord.SaveInMongoFunction(RouteName,
                                              os.path.join(TransportModeDir, RouteName, fileName),
                                              f'{DeviceID}.{Mode}')


Aforementioned, in earlier unit, the `horizontal and vertical components` of accelerometer records for the bus, bike and car trips corresponding to the `stoppage segments` resembles similar pattern. Therefore, such stoppage segments are discraded during the transport mode detection using `stopage triggers`. Thus, the transport mode detector is trained and validated on the `horizontal and vertical components` of accelerometer records of segments other than stoppage segment based on the `accelerometer triggers`.

Further, we have trained the `transport mode detector` for the `horizontal and vertical components` of accelerometer records of entire trip (including the stoppage segments). Subsequently, the performance of the former and latter approach are compared.

## Processing: For entire trip
This section describes the procedure for training the classifier on the `entire trip record` (including the stoppage semgents).

## Decomposition of accelerometer data
The accelerometer record data is decomposed to `horizontal and vertical components` with the `IntervalLength` of `140 points.`

In [6]:
def ComputeEarthAxisComponentRaw(RouteName, IntervalLength, RecordType):
    '''
    input: The route name, interval length for computing accelerometer components
    and record type variable to specify the segment type of the records.
    
    output: None
    
    function: It extracts the raw accelerometer records from the MongoDB database
    for the entire trip records and executes the code to compute the horizontal and
    vertical components based on the method proposed in Jigsaw paper. 
    '''
    SingleTripsInfo = [LR['SingleTripInfo'] for LR in 
                       con[RouteName]['TripsInfo'].find({'ConvertedToEarthAxis':False})]
    for SingleTripInfo in SingleTripsInfo:
        print(SingleTripInfo)
        AcclMagRecord = [collection for collection in 
                         con[RouteName][SingleTripInfo+'.AcclMagData.Raw'].find().sort([('GPSIndex',1)])]

        EarthaxisAccelerationOnRawRecords.ProcessEarthaxisHVComponentUsingJigSawMethod(RouteName,
                                                                                       SingleTripInfo,
                                                                                       AcclMagRecord,
                                                                                       IntervalLength,RecordType)
        con[RouteName]['TripsInfo'].update_one(
            {'SingleTripInfo':SingleTripInfo},{'$set':{'ConvertedToEarthAxisRaw':True}})

    '''Index'''
    for index in range(len(SingleTripsInfo)):
        con[RouteName][SingleTripsInfo[index]+".EAccHVComponent.Raw"+""].create_index('GPSIndex')
   

## Feature Extraction

The feature `set-1 to set-4` are computed using the orientation independent horizontal and vertical components of accelerometer records for the window of `128 samples and 50% overlap`.

In [7]:
def ComputeFeatures_Raw(ProjectDataUsed, WindowList, WindowIndex, RecordType, RouteName):
    
    '''
    input: The ProjectDataUsed variable specify whether the project dataset should to use project 
    dataset or the user collected dataset. The other variables specify the window size for feature
    computation, record type (Raw or Segment other than stoppage segment), and route name.
    
    output: None
    
    function: It extracts the appropriate accelerometer components from the MongoDB database based
    on the provided input and computes the features on the windowed accelerometer component. The 
    computed features are stored in the MongoDB database.
    '''
    
    if ProjectDataUsed==True:
        PDCar, PDBike, BMSingleTripsInfoList, CMSingleTripsInfoList = TransportModeFeatureHelper.GetTripsForTransportModes()

        TransportModeFeatureHelper.GetFeaturesForGivenTripType (BMSingleTripsInfoList, 0,
                                                            WindowList[WindowIndex],RecordType, RouteName)
        TransportModeFeatureHelper.GetFeaturesForGivenTripType (CMSingleTripsInfoList, 0,WindowList[WindowIndex],
                                                                RecordType, RouteName)
        TransportModeFeatureHelper.GetFeaturesForGivenTripType (PDCar, 1,WindowList[WindowIndex],
                                                                RecordType, RouteName)
        TransportModeFeatureHelper.GetFeaturesForGivenTripType (PDBike, 2,WindowList[WindowIndex],
                                                                RecordType, RouteName)

    else:
        Trips = SingleTripsInfo = [LR['SingleTripInfo'] for LR in con[RouteName]['TripsInfo'].find(
            {'ConvertedToEarthAxisRaw':True})]

        BusTrips = [Trip for Trip in Trips if Trip.split['_'][1]=='Bus']
        CarTrips = [Trip for Trip in Trips if Trip.split['_'][1]=='Car']
        BikeTrips = [Trip for Trip in Trips if Trip.split['_'][1]=='Bike']

        TransportModeFeatureHelper.GetFeaturesForGivenTripType (BusTrips, 0,
                                                                WindowList[WindowIndex],RecordType, RouteName)
        TransportModeFeatureHelper.GetFeaturesForGivenTripType (CarTrips, 1,
                                                                WindowList[WindowIndex],RecordType, RouteName)
        TransportModeFeatureHelper.GetFeaturesForGivenTripType (BikeTrips, 2,
                                                                WindowList[WindowIndex],RecordType, RouteName)        

## Processing: For Records other than Stoppage Segment
This section describes the procedure for training the classifier on the `segments other than stoppage segment`.

## Decomposition of accelerometer data
The accelerometer record data is decomposed to `horizontal and vertical components` with the `IntervalLength` of `140 points.`

In [8]:
def ExtractSegment_EarthAxis_OtherThanStoppages(ProjectDataUsed, RouteName, Speed_H = 5.6, Speed_L = 3):
    
    '''
    input: The route name, interval length for computing accelerometer components, 
    speed range for computing segment other than stoppage segments and record type
    variable to specify the segment type of the records.
    
    output: None
    
    function: It extracts the raw accelerometer records from the MongoDB database
    for the computing segment other than stoppage segments and executes the code to
    compute the horizontal and vertical components based on the method proposed in 
    Jigsaw paper. 
    '''
    
    '''Extract segment other than stoppage segments'''
    if ProjectDataUsed==True:
        BMSingleTripsInfoList,CMSingleTripsInfoList,PDSingleTripsInfoList = SegmentsOtherThanStoppageSegments.GetTripsBasedOnType(RouteName)
        SegmentsOtherThanStoppageSegments.GetBMAndPDSegments(RouteName,BMSingleTripsInfoList,Speed_H, Speed_L)
        SegmentsOtherThanStoppageSegments.GetBMAndPDSegments(RouteName,PDSingleTripsInfoList,Speed_H, Speed_L)
        SegmentsOtherThanStoppageSegments.GetCMSegments(RouteName,CMSingleTripsInfoList,BMSingleTripsInfoList)
        SegmentsOtherThanStoppageSegments.GetGPSAndAcclReadOfSegment(RouteName)

    else:
        SingleTripsInfo = [LR['SingleTripInfo'] for LR in con[RouteName]['TripsInfo'].find({'SegmentExtracted':False})]
        SegmentsOtherThanStoppageSegments.GetBMAndPDSegments(RouteName,SingleTripsInfo, Speed_H, Speed_L)
        SegmentsOtherThanStoppageSegments.GetGPSAndAcclReadOfSegment(RouteName)

    '''
    for SingleTripInfo in SingleTripsInfo:
        con[RouteName]['TripsInfo'].update_one(
            {'SingleTripInfo':SingleTripInfo},{'$set':{'ConvertedToEarthAxis':False}})
    '''
    
    EarthaxisAcceleration.ConvertToEarthaxisAcc(RouteName,RecordType)

## Features Extraction

In [9]:
def ComputeFeatures_SegmentOtherThanStoppages(ProjectDataUsed, WindowList, WindowIndex, RecordType, RouteName):
    
    '''
    input: The ProjectDataUsed variable specify whether the project dataset should to use project 
    dataset or the user collected dataset. The other variables specify the window size for feature
    computation, record type (Raw or Segment other than stoppage segment), and route name.
    
    output: None
    
    function: It extracts the appropriate accelerometer components from the MongoDB database based
    on the provided input and computes the features on the windowed accelerometer component. The 
    computed features are stored in the MongoDB database.
    '''
    
    '''Recent Attempt'''

    if ProjectDataUsed==True:
        PDCar, PDBike, BMSingleTripsInfoList, CMSingleTripsInfoList = TransportModeFeatureHelper.GetTripsForTransportModes()

        TransportModeFeatureHelper.ExtractFeaturesOfGivenTypeOfTrip(BMSingleTripsInfoList,"Bus",0,
                                                                    WindowList[WindowIndex],RecordType, RouteName)
        TransportModeFeatureHelper.ExtractFeaturesOfGivenTypeOfTrip(CMSingleTripsInfoList,"Bus",0,
                                                                    WindowList[WindowIndex],RecordType, RouteName)
        TransportModeFeatureHelper.ExtractFeaturesOfGivenTypeOfTrip(PDCar,"Car",1,WindowList[WindowIndex],
                                                                    RecordType, RouteName)
        TransportModeFeatureHelper.ExtractFeaturesOfGivenTypeOfTrip(PDBike,"Bike",2,WindowList[WindowIndex],
                                                                    RecordType, RouteName)    

    else:
        Trips = [LR['SingleTripInfo'] for LR in con[RouteName]['TripsInfo'].find(
            {'ConvertedToEarthAxis':True})]

        BusTrips = [Trip for Trip in Trips if Trip.split['_'][1]=='Bus']
        CarTrips = [Trip for Trip in Trips if Trip.split['_'][1]=='Car']
        BikeTrips = [Trip for Trip in Trips if Trip.split['_'][1]=='Bike']

        TransportModeFeatureHelper.ExtractFeaturesOfGivenTypeOfTrip(BusTrips,"Bus",0,
                                                                    WindowList[WindowIndex],RecordType, RouteName)
        TransportModeFeatureHelper.ExtractFeaturesOfGivenTypeOfTrip(CarTrips,"Car",1,WindowList[WindowIndex],
                                                                    RecordType, RouteName)
        TransportModeFeatureHelper.ExtractFeaturesOfGivenTypeOfTrip(BikeTrips,"Bike",2,WindowList[WindowIndex],
                                                                    RecordType, RouteName)

The feature `set-1 to set-4` are computed using the orientation independent horizontal and vertical components of accelerometer records for the window of `128 samples and 50% overlap`.

## For saving Mongo data in numpy

In [10]:
def LoadInMongoFromNp(RouteName, NpPathDir):

    '''
    input: The route name and numpy directory path
    output: The MongoDB database collections from the Numpy files
    function: It creates the MongoDB database of TripsInfo collections 
    and features collections from the Numpy files.
    '''    
    
    CollectionName = 'TripsInfo'
    TripsInfoRecords = np.load(f'{NpPathDir}/{RouteName}/{CollectionName}.npy', allow_pickle=True)
    
    print('Saving data in mongoDB')
    print(RouteName, CollectionName)
    con[RouteName][CollectionName].insert_many(TripsInfoRecords.tolist())
    
    CollectionNames = os.listdir(f'{NpPathDir}/{RouteName}')
    #print(CollectionNames)
    
    CollectionNames_1 = [rec for rec in CollectionNames if '\'.' not in rec] # To address the error
    CollectionNames_2 = [rec for rec in CollectionNames if 'Feature' in rec]
    
    #print('Saving data in mongoDB')
    for Collection in CollectionNames_2:
        RecordsList = np.load(f'{NpPathDir}/{RouteName}/{Collection}', allow_pickle=True)
        #print(Collection)
        #pprint.pprint(RecordsList[0:3])
        
        CollectionName = Collection[0:-4]
        
        print(RouteName, CollectionName)
        con[RouteName][CollectionName].insert_many(RecordsList.tolist())
        
    
    
def SaveInNp(RouteName, NpPathDir):
    
    '''
    input: The route name and numpy directory path
    output: Numpy files of the MongoDB database
    function: It stores the Numpy files for the MongoDB database in the specified directory path
    '''
    
    CollectionNames = [Collection for Collection in 
                        con[RouteName].list_collection_names() if Collection!='system.indexes']

    for CollectionName in CollectionNames:
        print('CollectionName', CollectionName)
        RecordsList = [rec for rec in con[RouteName][CollectionName].find().sort([('_id',1)])]

        for RecordDict in RecordsList:
            del[RecordDict['_id']]

        if os.path.exists(os.path.join(NpPathDir, RouteName)) == False:
            os.mkdir(os.path.join(NpPathDir, RouteName))

            
        #np.save(f'{Path}/{Database}/{CollectionName}.npy', RecordsList)
        np.save(os.path.join(NpPathDir, RouteName,f'{CollectionName}.npy'), RecordsList)
        

## Execution of Data extraction, accl. component, and feature computation for `.Raw` and `SegmentOtherThanStoppages`

### Parameters

In [11]:
path = Path(os.getcwd())
OneLevelUpPath = path.parents[0]

In [12]:
IntervalLength = 160

'''Trips and Window size for Feature extraction'''
#BMSingleTripsInfoList,CMSingleTripsInfoList,PDSingleTripsInfoList = TransportModeFeatureHelper.GetTripsBasedOnType(RouteName)
WindowList = [32,64,128,256,512]
WindowIndex = 2
WindowSize = WindowList[WindowIndex]

'''For segment other than stoppages'''
Speed_H = 5.6
Speed_L = 3

'''Variables for Classification'''
ResultPathDir = os.path.join(str(OneLevelUpPath), 'results','Transport','')


if os.path.exists(ResultPathDir) == False:
    os.mkdir(ResultPathDir)


TrainedModelPathDir = os.path.join(str(OneLevelUpPath), 'data', 'TrainedModel','Transport','')

'''Path for Np'''
NpPathDir = os.path.join(str(OneLevelUpPath), 'data','NpData')

'''Path for Model'''
ClassifierList = [GaussianNB(), LogisticRegression(random_state=0), 
                  RandomForestClassifier(max_depth=20), tree.DecisionTreeClassifier(), 
                  SVC(gamma='auto')
                 ]

ClassifierNameList = ['NB', 'LogisticRegression', 'RF', 'DT', 'SVC']

## Variables

`ProjectDataUsed`: determines whether the project data or the user's own data is used for execution.

`UsedPreTrained`: determines whether the pretrained and precomputed dataset or raw data is used for execution.

`UseMongoDB`: determines whether the MonngoDB database or Numpy file is used for execution.

`ReducedKFolds`: If false: one fold is used, else ten-fold is used

In [13]:
'''
ProjectDataUsed = True
UsedPreTrained = True
ReducedKFolds = False
UseMongoDB = True
'''

#'''
ProjectDataUsed = True
UsedPreTrained = True
ReducedKFolds = True
UseMongoDB = False
#'''

'''
ProjectDataUsed = True
UsedPreTrained = False
ReducedKFolds = False
UseMongoDB = False
'''

In [ ]:
#TransportModeDir = '/home/pruthvish/JRF/GitVersion_PMC/Data/TransportMode/'
if ProjectDataUsed==True:
    TransportModeDir = os.path.join(str(OneLevelUpPath), 'data', 'TransportMode','')
else:
    TransportModeDir = os.path.join(str(OneLevelUpPath), 'data', 'UserData', 'TransportMode','')
RouteNamesList = ['ISCON_PDPU_For_Transport_Mode']
#RouteNamesList = [f for f in os.listdir(TransportModeDir)]

### Code

In [14]:
for RouteName in RouteNamesList:
    if UsedPreTrained==False and UseMongoDB==True:
        #'''
        print('Reading data and saving in MongoDB')
        ReadAndSaveData(TransportModeDir, RouteName)

        RecordType = '.Raw'
        print(f'Computing preprocessing for {RecordType} segments')
        ComputeEarthAxisComponentRaw(RouteName, IntervalLength, RecordType)
        print(f'Computing features for {RecordType} segments')
        ComputeFeatures_Raw(ProjectDataUsed, WindowList, WindowIndex, RecordType, RouteName)
        #'''

        RecordType = '.SegmentOtherThanStoppage'
        
        print(f'Computing preprocessing for {RecordType} segments')
        ExtractSegment_EarthAxis_OtherThanStoppages(ProjectDataUsed, RouteName, Speed_H = 5.6, Speed_L = 3)
        print(f'Computing features for {RecordType} segments')
        ComputeFeatures_SegmentOtherThanStoppages(ProjectDataUsed, WindowList, WindowIndex, RecordType, RouteName)

        print('Saving MongoData in Np files')
        SaveInNp(RouteName, NpPathDir)
    
    elif UseMongoDB==True:
        
        RouteNamesListInDB = con.list_database_names()
        if RouteName not in RouteNamesListInDB:
            '''Load the data for RouteName, if RouteName is not in RouteNamesList'''
            print('Loading MongoData from Np files')
            LoadInMongoFromNp(RouteName, NpPathDir)


## Classification

In [15]:
def GetFeaturesForClass(RouteNameTestList, FeatureType, RecordType,
                        SelectedFeatures, SelectedFeaturesFlag, NpPathDir, UseMongoDB):
    if UseMongoDB==True:
        
        if ProjectDataUsed==True:
            X, y = TMC_Helper.GetData(FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag)
        else:
            First_InputFlag = True
            for RouteNameTest in RouteNameTestList:
                X_Route, y_Route = TMC_Helper.GetData_User(RouteNameTest, FeatureType, RecordType,
                                                           SelectedFeatures, SelectedFeaturesFlag)

                if First_InputFlag==True:
                    X = X_Route
                    y = y_Route
                    First_InputFlag = False

                else:
                    X = np.concatenate((X, X_Route))
                    y = np.concatenate((y, y_Route))
                    '''
                    X += X_Route
                    y += y_Route
                    '''

                print('X.shape', X.shape)
                print('y.shape', y.shape)
        
        
        '''Save in NpData'''
        
        if os.path.exists(os.path.join(NpPathDir,'Transport'))==False:
            os.mkdir(os.path.join(NpPathDir,'Transport'))
            
        np.save(f'{NpPathDir}/Transport/X_{FeatureType}_{RecordType}_{SelectedFeaturesFlag}.npy', X)
        np.save(f'{NpPathDir}/Transport/y_{FeatureType}_{RecordType}_{SelectedFeaturesFlag}.npy', y)
        
    else:
        X = np.load(f'{NpPathDir}/Transport/X_{FeatureType}_{RecordType}_{SelectedFeaturesFlag}.npy',
                    allow_pickle=True)
        y = np.load(f'{NpPathDir}/Transport/y_{FeatureType}_{RecordType}_{SelectedFeaturesFlag}.npy',
                    allow_pickle=True)
    
    return(X, y)

In [16]:
def ApplyClassification(Modes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                        Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNameTestList,
                        UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds,
                        NpPathDir, UseMongoDB
                       ):
    '''
    input: 
        Modes: The number of classes
        
        FeatureType and RecordType: feature type and record type information
        
        SelectedFeatures and SelectedFeaturesFlag: selected features list and flag to 
        determine if selected features are list is used
        
        Classifier and ClassifierName: classifier object variable and classifier name
        
        ResultPathDir: path of result directory
        
        ProjectDataUsed: flag to determine whether the project dataset or user dataset is used
        
        RouteNameTestList: The route names list to be considered during classification
        
        UsedPreTrained: Flag to determine whether the pretrained classifier should be used
        or the classifier is to be trained
        
        ClassifierName_ForModelSave: the name of classifier for saving a model
        
        TrainedModelPathDir: path of trainedModel directory 
    
    output: None
    
    function: It trains and validates the classifier the ten-fold cross-validation with stratified
    sampling of each class. The performance metrics of the classifier is stores as a txt file in the
    result directory
    
    '''
    
    MetricsDict = TMC_Helper.InitializeMetricsDict(Modes)
    
    
    X, y = GetFeaturesForClass(RouteNameTestList, FeatureType, RecordType,
                               SelectedFeatures, SelectedFeaturesFlag, NpPathDir, UseMongoDB)    

            
    MetricsDict = TMC_Helper.TrainAndPredict(X, y, Classifier, MetricsDict, ResultPathDir,
                                             ClassifierName, UsedPreTrained, 
                                             ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds)
    
    TMC_Helper.PrintMetricsDict(ClassifierName, ResultPathDir, FeatureType, RecordType,
                                SelectedFeaturesFlag, MetricsDict)
    '''
    filename = os.path.join(ResultPathDir,f'{ClassifierName}_TrainedModel.sav')
    f'ResultPathDir{ClassifierName}'
    pickle.dump(model, open(, 'wb'))
    '''


In [17]:
Modes = 3

In [18]:
for Classifier, ClassifierName in zip(ClassifierList, ClassifierNameList):
    for RecordType in ['.Raw', '.SegmentOtherThanStoppage']:
        
        RecordTypeName = RecordType.split('.')[-1]
        
        '''Feature set-1'''
        FeatureType = '.HARFeature'
        SelectedFeaturesFlag = False
        SelectedFeatures = []
        ClassifierName_ForModelSave = f'{ClassifierName}_{RecordTypeName}_Set1'
        print('Feature set-1')
        ApplyClassification(Modes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                            Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNamesList, 
                            UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds,
                            NpPathDir, UseMongoDB
                           )

        '''Feature set-2'''
        FeatureType = '.HARFeature'
        SelectedFeaturesFlag = True
        SelectedFeatures = TMC_Helper.SelectedFeaturesForFeatureType(FeatureType)
        ClassifierName_ForModelSave = f'{ClassifierName}_{RecordTypeName}_Set2'
        print('Feature set-2')
        ApplyClassification(Modes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                            Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNamesList,
                            UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds,
                            NpPathDir, UseMongoDB
                           )

        '''Feature set-3'''
        FeatureType = '.TransportFeatures'
        SelectedFeatures = []
        SelectedFeaturesFlag = False
        ClassifierName_ForModelSave = f'{ClassifierName}_{RecordTypeName}_Set3'
        print('Feature set-3')
        ApplyClassification(Modes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                            Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNamesList,
                            UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds,
                            NpPathDir, UseMongoDB
                            
                           )   


        '''Feature set-4'''
        FeatureType = '.TransportFeatures'
        SelectedFeatures = TMC_Helper.SelectedFeaturesForFeatureType(FeatureType)
        SelectedFeaturesFlag = True
        ClassifierName_ForModelSave = f'{ClassifierName}_{RecordTypeName}_Set4'
        print('Feature set-4')
        ApplyClassification(Modes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                            Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNamesList,
                            UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds,
                            NpPathDir, UseMongoDB
                           )        
        
        print('Classifier, ClassifierName', Classifier, ClassifierName)

Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName GaussianNB() NB
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName GaussianNB() NB
Feature set-1


/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Feature set-2


/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Feature set-3


...

Feature set-3


/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Feature set-4


/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Classifier, ClassifierName LogisticRegression(random_state=0) LogisticRegression
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName RandomForestClassifier(max_depth=20) RF
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName RandomForestClassifier(max_depth=20) RF
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName DecisionTreeClassifier() DT
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName DecisionTreeClassifier() DT
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName SVC(gamma='auto') SVC
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName SVC(gamma='auto') SVC


In [19]:
'''Read the value for one of the machine learning algorithm'''
file = os.path.join(ResultPathDir,f'{ClassifierName}.txt')
f = open(file, "r")
print(f.read())

Results for .Raw, .HARFeature, and selected features flag: False
ConfusionMatrix 
[[1245.  134.  212.]
 [   2. 1754.   11.]
 [ 135.   11. 1865.]]
 
PrecissionValue 
[0.90086831 0.92364402 0.89319923]
 
RecallValue 
[0.78252671 0.9926429  0.9273993 ]
 
F1ScoreValue 
[0.83753784 0.95690125 0.90997804]
 
AccuracyValue 
0.9059415161110076
 
Results for .Raw, .HARFeature, and selected features flag: True
ConfusionMatrix 
[[1266.  135.  190.]
 [   4. 1753.   10.]
 [ 165.    8. 1838.]]
 
PrecissionValue 
[0.88222997 0.92457806 0.90186457]
 
RecallValue 
[0.79572596 0.99207697 0.91397315]
 
F1ScoreValue 
[0.83674818 0.95713896 0.90787849]
 
AccuracyValue 
0.9046377351462097
 
Results for .Raw, .TransportFeatures, and selected features flag: False
ConfusionMatrix 
[[ 972.   48.  744.]
 [  34. 1722.   11.]
 [ 145.   90. 1776.]]
 
PrecissionValue 
[0.84448306 0.92580645 0.70169893]
 
RecallValue 
[0.55102041 0.97453311 0.88314272]
 
F1ScoreValue 
[0.66689537 0.94954508 0.78203435]
 
AccuracyValue